# Timing

Our initial idea for using the Persistent Memory devices relies on obtaining timing information for the various kernels in an nGraph function with the inputs/outputs in different different memory pools, and using this timing to assign weights to an optimization routine.

The questions this notebook is trying to answer is:

1. If profiling individual kernels in this way provides somewhat consistent and reliable results.
2. If profiling multiple kernel configurations at a time across the graph yields similar results to testing one configuration at a time. This optimization comre from the large number of configurations that have to be tested for larger graphs. Overlapping thests for non-neighboring ops can speed up profiling time by over an order of magnitude.

The general pipeline for performing the profiling is:

1. Build a nGraph function + executable
2. Get all nodes in the graph that we want to test and enumerate all the configurations to be tested.
3. Pick a configuration to test and configure the graph to reflect that configuration. Optionally, keep greedily picking configurations that don't overlap until no more can be selected.
4. Recompile the graph with the new memory configuration.
5. Use the built-in timing features of nGraph codegen to profile the running times for internal nodes.
6. Repeat until all configurations have been checked.

In [3]:
using Pkg; Pkg.activate(".")

using Runner, Checkpoints

# Setup checkpoints
setdepot("./timing-checkpoints")

┌ Info: Precompiling Runner [4a6e9825-ed04-540b-82d3-f33d0e8d45fb]
└ @ Base loading.jl:1186
ERROR: LoadError: LoadError: could not load library "/home/mark/projects/nGraph/deps/usr/lib/libngraph.so"
libpmemobj.so.1: cannot open shared object file: No such file or directory
Stacktrace:
 [1] #dlopen#3(::Bool, ::Function, ::String, ::UInt32) at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.1/Libdl/src/Libdl.jl:109
 [2] dlopen at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.1/Libdl/src/Libdl.jl:109 [inlined] (repeats 2 times)
 [3] top-level scope at none:0
 [4] include at ./boot.jl:326 [inlined]
 [5] include_relative(::Module, ::String) at ./loading.jl:1038
 [6] include at ./sysimg.jl:29 [inlined]
 [7] include(::String) at /home/mark/projects/nGraph/src/nGraph.jl:1
 [8] top-level scope at none:0
 [9] include at ./boot.jl:326 [inlined]
 [10] include_relative(::Module, ::String) at ./loading.jl:1038
 [11] include(::Module, ::String) at ./sysimg.jl

ErrorException: Failed to precompile Runner [4a6e9825-ed04-540b-82d3-f33d0e8d45fb] to /home/mark/.julia/compiled/v1.1/Runner/F5BZU.ji.